# Number of photons in a resonator and T2E given drive and noise input powers

We ask the following questions:

1. If we have a noise with spectral density $N_0$ (units are dBm/Hz) incident on a readout resonator, what is the number of photons in the resonator?
2. If we have a coherent drive with power $P$ and frequency $\omega_d$ incident on a readout resonator, what is the number of photons?
3. In both cases above, what is the dephasing rate of a qubit coupled to the resonator?

Important sources of information:

1. A paper showing that the resonator noise spectrum can be directly measured with a spin locking technique, both for coherent and thermal noise sources: [Distinguishing coherent and thermal photon noise in a circuit QED system, F. Yan et al., 2018](https://arxiv.org/abs/1801.00467)
2. A paper with a practical measurement of the dephasing and AC stark shift as a function of number of photons, contains expresssions for both [The flux qubit revisited to enhance coherence and reproducibility, F. Yan et al., 2016](https://www.nature.com/articles/ncomms12964#MOESM137)
3. A classic paper with all derivations related to cavity noise and signal levels and input output theory: [Introduction to Quantum Noise, Measurement and Amplification, A. A. Clerk et al., 2010](https://arxiv.org/abs/0810.4729)

## Preamble: What is the power needed to put one photon in the resonator when driving on resonance?

Let's say we have a drive with power $P$ in dBm. Then at RT, in dBW (watts dB) it will be $P-30$. Then when going through the attenuation chain it will be attenuated by about 70dB, so it will reach the resonator with power 

$$
P_{dBW,20mK} = P_{dBm,RT} - 100
$$

How do we convert it to number of photons? We need to divide by $\hbar \omega_c$, but since $h$ is very small, this means we will need to add a lot.

In [44]:
# what is h*fc in dB for converting from energy to photon number?

fc = 7 # GHz
h = 6.626e-34 # J*s

print(10*np.log10(h*fc*1e9))

-233.33650528439802


this means that 

$$
\bar{n} = P_{dBm,RT} - 100 + 230
$$

so even a -130dBm spur on resonance will put one photon in the resonator!!! As we will see, the dephasing time is about $\chi\bar{n}$, so this is not tolerable, as one photon will lead to a dephasing time of about $1\rm{\mu sec}$.

## Question 3

We have (see https://www.nature.com/articles/ncomms12964#MOESM137 eq. 5):

$$
\Gamma_\phi^{th} = \frac{4\kappa\chi^2}{\kappa^2 + 4\chi^2}\bar{n}
$$
where $\chi$ is defined such that $2\chi/2\pi$ is the difference in freq between the e and g Lorenzians (see fig. 5a).

Let's first take the parameters from this paper:



In [24]:
import numpy as np

chi = 2*np.pi*0.9  # MHz
kappa = 2*np.pi*1.5  # MHz
nbar = 0.009

gamma_phi = 4*kappa*chi**2/(kappa**2+4*chi**2)*nbar
print(f'{gamma_phi=} MHz')
print(f'dephasing time is {1/gamma_phi:.2f} us')


gamma_phi=0.05005947638179145 MHz
dephasing time is 19.98 us


## Question 1

Now let's try to estimate the number of photons from the noise power. Hueristically, if we have a noise power in dBm/Hz, the number of photons should be roughly this divided by $\hbar \omega_c$. Let's try:

In [42]:
fc = 7 # GHz
h = 6.626e-34

N0_dBmHz_RT = -144 # dBm/Hz (room temperature)
Attenuation = -70 # dB
N0_dBmHz = N0_dBmHz_RT + Attenuation

N0 = 1e-3*10**(N0_dBmHz/10)  # W/Hz
print(f'{N0=:.2e} W/Hz')
N_photon = N0 / (h*fc*1e9)
print(f'{N_photon=:.2f} photons')

N0=3.98e-25 W/Hz
N_photon=0.09 photons


## Question 2

Now let's try to estimate the number of photons from an off-resonant drive with very high power.

The relation of the drive power to the number of qubits is given by the following derivation.

We have an equation of motion (in freq domain), see eq. E39 here: https://arxiv.org/pdf/0810.4729.pdf

$$
-i\omega_d a = -(\kappa/2)a -i\omega_c - \sqrt{\kappa} b
$$

We can take $b$ to be equal to the incoming photon flux (see eq. E73):

$$
b = -i \sqrt{\bar{\dot{N}}} = -i \sqrt{\frac{P}{\hbar \omega}}
$$

Thus the photon number is:

$$
\bar{n} = |a|^2 = \frac{\kappa}{(\kappa/2)^2 + \Delta^2}\frac{P}{\hbar\omega_d}
$$

If the detuning is about 1GHz, and kappa is about 1MHz, then this simplifies to:

$$
\bar{n} \simeq \frac{\kappa}{\Delta^2}\frac{P}{\hbar\omega_d}
$$
and so this $\frac{\kappa}{\Delta^2}$ prefactor is about -180+60 = -120 in dB.

Putting in numbers, we get:

In [43]:
fd = 9.4 # GHz
Pd_dBm_RT = 2 # dBm
crosstalk_loss = -20 # dB
Pd_dBm = Pd_dBm_RT + Attenuation + crosstalk_loss
Pd = 1e-3*10**(Pd_dBm/10)  # W
Pd_photon = Pd / (h*fd*1e9)
print(f'{Pd_photon=:.2e} photons/sec')
nbar = Pd_photon * kappa * 1e6 / (4*np.pi**2*1e18*(fd - fc)**2)
print(f'{nbar=:.4f} photons')

Pd_photon=2.54e+11 photons/sec
nbar=0.0105 photons
